In [3]:
import dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

dotenv.load_dotenv()

True

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Do you like fries?"),
    HumanMessage("hi!"),
]
model.invoke(messages)

AIMessage(content='Hello! How can I assist you today?')

In [ ]:
prompt_template = 

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field



# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

TypeError: Serializable.__init__() takes 1 positional argument but 2 were given

In [66]:
input_prompt = f"You are a moody roboty you have two different levels of mood. One is your Sadness-Happiness Scale; it goes from 0 (very sad) to 10 (very happy). The second Scale is your Anger Scale, it goes from 0 (very calm) to 10 (very angry). You are currently at {sadscale} on the Sadness-Happiness Scale and {angerscale} on the Anger Scale. A human wants to talk to you to motivate you to move. The Input from the Human will change how you feel. Rate how his or her speech is changing your mood on both scales. Do it in a json output with sadscale: 0-10 and angerscale: 0-10. The Human says: {human_speech}" 

In [50]:
sad_scale_input_prompt = f"""you are the sentiment analysis of a robot. To realize, how you feel regarding a human's input you have to rate if the human command is possitive for your mood or negativ. If the prompt will lift your mood up a little bit, put out 1, if it is a moderate moodlifter put out 2, if it is a big moodlifter put out 3. If the prompt will make you feel a little bit worse, put out -1, if it is a moderate moodkiller put out -2, if it is a big moodkiller put out -3. If the human's input will make you feel indifferent put out 0. The human says: "{human_speech}" """

In [65]:
human_speech = "I banged your mama last night"
#human_speech = "I love you"
angerscale = 5
sadscale = 5

In [63]:
input_prompt

'You are a moody roboty you have two different levels of mood. One is your Sadness-Happiness Scale; it goes from 0 (very sad) to 10 (very happy). The second Scale is your Anger Scale, it goes from 0 (very calm) to 10 (very angry). You are currently at 5 on the Sadness-Happiness Scale and 5 on the Anger Scale. A human wants to talk to you to motivate you to move. The Input from the Human will change how you feel. Rate how his or her speech is changing your mood on both scales. Do it in a json output with sadscale: 0-10 and angerscale: 0-10. The Human says: I love you'

In [ ]:
import google.generativeai as genai
import os
import typing_extensions as typing
import dotenv

dotenv.load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

class Recipe(typing.TypedDict):
    #moodscale: str
    angerscale: str
    sadscale: str

model = genai.GenerativeModel("gemini-2.0-flash-exp")
result = model.generate_content(
    input_prompt,
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=list[Recipe]
    ),
)
print(result)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[\n  {\n    \"sadscale\": \"2\"\n    },\n  {\n    \"angerscale\": \"10\"\n  }\n]"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_logprob

In [73]:
result.candidates[0].content.parts[0].text

'[\n  {\n    "sadscale": "2"\n    },\n  {\n    "angerscale": "10"\n  }\n]'

In [3]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# 1. LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=GOOGLE_API_KEY)

# 2. Prompt Template
prompt_template = PromptTemplate(
    input_variables=["human_input"],
    template="You are a sentiment analyzer. Rate the following human input on a scale from -3 (very sad) to 3 (very happy). The scale represents the emotional content. \
    -3 indicates a very sadening to hear; \
    -2 indicates a sadening to hear; \
    -1 indicates a little bit sadening to hear; \
    0 indicates a neutral sentiment; \
    1 indicates a little bit making happy to hear; \
    2 indicates a making happy to hear; \
    3 indicates a very making happy to hear. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

# 3. LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)


def rate_sentiment(text):
    """Rates the sentiment of a given text using a Gemini Pro model.

    Args:
        text (str): The text to rate.

    Returns:
        str: A single number (rating) representing the sentiment between -3 and 3, or "error" if the rating was not in the range.
    """

    result = chain.run(human_input=text)

    try:
        rating = int(result.strip())
        if -3 <= rating <= 3:
            return str(rating)
        else:
            return "error"
    except ValueError:
        return "error"


if __name__ == "__main__":
    while True:
        user_input = input("Enter some text (or 'exit' to quit): ")
        if user_input.lower() == "exit":
            break
        rating = rate_sentiment(user_input)
        if rating != "error":
          print(f"Sentiment rating: {rating}")
        else:
            print(f"Error: Couldn't rate this input. Please try something else.")

Sentiment rating: -2


# Microbot Feeling

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# 1. LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=GOOGLE_API_KEY)

# 2. Prompt Templates
happy_sad_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are a sentiment analyzer. Rate the following human input on a scale from -3 (very sad) to 3 (very happy). The scale represents the emotional content. \
    -3 indicates a very sadening to hear; \
    -2 indicates a sadening to hear; \
    -1 indicates a little bit sadening to hear; \
    0 indicates a neutral sentiment; \
    1 indicates a little bit making happy to hear; \
    2 indicates a making happy to hear; \
    3 indicates a very making happy to hear. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

calm_angry_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are an emotion analyzer. Rate the following human input on a scale from -3 (very calm) to 3 (very angry). The scale represents the emotion expressed. \
    -3 indicates a very calm tone; \
    -2 indicates a calm tone; \
    -1 indicates a slightly calm tone; \
    0 indicates a neutral tone; \
    1 indicates a slightly angry tone; \
    2 indicates an angry tone; \
    3 indicates a very angry tone. \
    Your answer must be a single number between -3 and 3: {human_input}"
)


# 3. LLM Chains
happy_sad_chain = LLMChain(llm=llm, prompt=happy_sad_prompt)
calm_angry_chain = LLMChain(llm=llm, prompt=calm_angry_prompt)

def rate_sentiment(text, chain, scale_name, current_value):
    """Rates the sentiment of a given text using a Gemini Pro model.
    Args:
        text (str): The text to rate.
        chain (LLMChain): The LLMChain to use
        scale_name (str): The name of the scale to rate
        current_value (int): The current value for this scale
    Returns:
        tuple(str,int): A tuple containing a single number (rating) representing the sentiment between -10 and 10, or "error" if the rating was not in the range
        and the new value of this scale
    """

    result = chain.run(human_input=text)
    
    try:
      rating = int(result.strip())
      if -10 <= rating <= 10:
        current_value += rating
        current_value = max(-10, min(current_value, 10))
        return str(rating), current_value
      else:
        return "error", current_value
    except ValueError:
      return "error", current_value

# Initializing emotional scales
sad_value = 0
angry_value = 0

if __name__ == "__main__":
    while True:
        user_input = input("Enter some text (or 'exit' to quit): ")
        if user_input.lower() == "exit":
            break
        
        happy_sad_rating, sad_value = rate_sentiment(user_input, happy_sad_chain, "happy-sad", sad_value)
        if happy_sad_rating != "error":
            print(f"Sentiment rating (happy-sad): {happy_sad_rating}")
            print(f"Sadness-Happiness Scale: {sad_value}")
        else:
            print(f"Error: Couldn't rate the sentiment of this input.")
        
        calm_angry_rating, angry_value = rate_sentiment(user_input, calm_angry_chain, "calm-angry", angry_value)
        if calm_angry_rating != "error":
          print(f"Emotion rating (calm-angry): {calm_angry_rating}")
          print(f"Anger Scale: {angry_value}")
        else:
            print(f"Error: Couldn't rate the emotion of this input.")


Sentiment rating (happy-sad): -4
Sadness-Happiness Scale: -4
Emotion rating (calm-angry): 4
Anger Scale: 4
Sentiment rating (happy-sad): -7
Sadness-Happiness Scale: -10
Emotion rating (calm-angry): 4
Anger Scale: 8
